# Introduction

- Following code is for retrieving data of iitk students from the iitk server using their roll number. <br>
- In roll_no_list, append all the roll numbers whose name and email you want to retrieve from the server. <br>
- If you are outside from IIT Kanpur, then don't forget to use ssl-vpn proxy.
<br><br>
Change Gmail settings - https://myaccount.google.com/lesssecureapps

## Import Libraries

In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import requests
import re
import pandas as pd

import email
import smtplib
import ssl

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [16]:
# Save roll numbers in roll_no_list

roll_no_list = []

# for y19
for i in range(1000):
    roll = 190001 + i
    roll_no_list.append(roll)

roll_no_list.append(15807678)

## Extracting Data from IITK Server

In [17]:
s = requests.Session()
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/Main_Frameset.jsp")
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/Main_Intro.jsp?frm='SRCH'")
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITK_Srch.jsp?typ=stud")


headers1 = {
    "Referer": "https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITk_SrchStudRoll_new.jsp"
}

payload1 = {
    'typ': ['stud'] * 12,
    'numtxt': '',
    'sbm': ['Y'] * 12
}

In [1]:
record = []
# roll_no_list = [15807678]
for roll in roll_no_list:
    payload1['numtxt'] = roll
    r1 = s.post("https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITk_SrchRes_new.jsp", headers=headers1, data=payload1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')

    name = ''
    email = ''
    department = ''
    gender = ''

    for para in soup1.select('.TableContent p'):
        body = para.get_text().strip()
        field = body.split(':')
        key = field[0].strip()
        value = field[1].strip()
        if key == 'Name':
            name = value.lower().title()
        elif key == 'E-Mail':
            email = value
        elif key == 'Department':
            department = value
        elif key == 'Gender':
            gender = value[0]

    if name != "" and email != "" and email != "Not Available":
        data = [name, email, department, gender, roll]
        record.append(data)
        user_name = name.split(" ")[0]
#         print (name, email, department, gender, roll)

df = pd.DataFrame(record, columns = ['Name', 'Email', 'Department', 'Gender', 'Roll No']) 
df.to_csv('y19_students.csv', encoding='utf-8', index=False)


# Sending mail to recipients

In [5]:
def message_template(user_name):
    message = """\
    
    
    <html>
    <body link="#00a5b5" vlink="#00a5b5" alink="#00a5b5" style="margin-top: 20px; margin-bottom: 20px;">
    <div class="s12">
        <table class=" main contenttable" align="center" style="font-weight: normal;border-collapse: collapse;border: 0;margin-left: auto;margin-right: auto;padding: 0;font-family: Arial, sans-serif;color: #555559;background-color: white;font-size: 16px;line-height: 26px;width: 600px;">
            <tr>
                <td class="border" style="border-collapse: collapse;border: 1px solid #eeeff0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;">
                    <table style="font-weight: normal;border-collapse: collapse;border: 0;margin: 0;padding: 0;font-family: Arial, sans-serif;">
                        <tr>
                            <td colspan="4" valign="top" class="image-section" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #fff;font-family: Arial, sans-serif;font-size: 20px;line-height: 26px;background: linear-gradient(45deg, #1de099, #1dc8cd);border-bottom: 0px solid #00a5b5">
                                <h3 style="margin: 20px; text-align: center;">UnchaAI</h3>
                            </td>
                        </tr>
                        <tr>
                            <td valign="top" class="side title" style="border-collapse: collapse;border: 0;margin: 0;padding: 20px;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;vertical-align: top;background-color: white;border-top: none;">
                                <table style="font-weight: normal;border-collapse: collapse;border: 0;margin: 0;padding: 0;font-family: Arial, sans-serif;">
                                    <tr>
                                        <td class="head-title" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 28px;line-height: 34px;font-weight: bold; text-align: center;">
                                            <div class="mktEditable" id="main_title">
                                                Paid Internship Program at UnchaAI
                                            </div>
                                        </td>
                                    </tr>
                                    <tr>
                                        <td class="top-padding" style="border-collapse: collapse;border: 0;margin: 0;padding: 15px 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 21px;">
                                            <hr size="1" color="#eeeff0">
                                        </td>
                                    </tr>
                                    <tr>
                                        <td class="text" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;">
                                        <div class="mktEditable" id="main_text">
                                            Hi {name},<br><br>
                                            We have internship offer for you, where you have to teach students. If you are interested, please fill the following form:<br><br>
                                            <div style="text-align: center; font-weight: bold;"><a href="https://reynsiatech.github.io/" style="color: #1e88e5;">Click here to apply</a></div><br>
                                            Have doubts? Feel free to contact us; we&apos;ll be glad to hear from you!
                                        </div>
                                        </td>
                                    </tr>
                                    <tr>
                                        <td style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 24px;">
                                         &nbsp;<br>
                                        </td>
                                    </tr>
                                </table>
                            </td>
                        </tr>
                        <tr bgcolor="#fff" style="border-top: 4px solid #00a5b5;">
                            <td valign="top" class="footer" style="border-collapse: collapse;border: 0;margin: 0;padding: 0;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 16px;line-height: 26px;background: #fff;text-align: center;">
                                <table style="font-weight: normal;border-collapse: collapse;border: 0;margin: 0;padding: 0;font-family: Arial, sans-serif;">
                                    <tr>
                                        <td class="inside-footer" align="center" valign="middle" style="border-collapse: collapse;border: 0;margin: 0;padding: 20px;-webkit-text-size-adjust: none;color: #555559;font-family: Arial, sans-serif;font-size: 12px;line-height: 16px;vertical-align: middle;text-align: center;width: 580px;">
                                        <div id="address" class="mktEditable">
                                            <b>UnchaAI</b><br>
                                            Reynsia Tech Pvt Ltd.<br>
                                            <a style="color: #00a5b5;" href="https://reynsiatech.github.io/">Contact Us</a>
                                        </div>
                                        </td>
                                    </tr>
                                </table>
                            </td>
                        </tr>
                    </table>
                </td>
            </tr>
        </table>
    </div>
    </body>
    </html>
    
    
    """.format(name = user_name)
    
    return message

In [13]:
def send_mail(user_name, user_mail_id):

    email_admin = 'reynsiatech@gmail.com'
    admin_password = 'Unchaaikijeetho'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_admin, admin_password)

    msg = MIMEMultipart()
    msg['From'] = "Reynsia Tech"
    msg['To'] = user_mail_id
    msg['Subject'] = "Work from home Internship Opportunity"

    message = message_template(user_name)

    msg.attach(MIMEText(message, 'html'))
    text = msg.as_string()
    server.sendmail(email_admin, user_mail_id, text)

    server.quit()

In [15]:
user_name = "Shivam Pal"
email = "shivam.pal128@gmail.com"
send_mail(user_name, email)

user_name = "Shivam Pal"
email = "shivampa@iitk.ac.in"
send_mail(user_name, email)

user_name = "Deepak Kumar"
email = "dk24pk@gmail.com"
send_mail(user_name, email)

user_name = "Piyush Patel"
email = "dapiyushpatel@gmail.com"
send_mail(user_name, email)